In [1]:
# Import all the necessary packages
#import uproot  # check out the documentation about the basics at https://uproot.readthedocs.io/en/latest/basic.html
import os
import re
import glob
import matplotlib.pyplot as plt
import awkward as ak
import datetime as dt
import numpy as np
import vector
import itertools
vector.register_awkward()

In [2]:
#path to parquet files
DATA_DIR = "."
PATHS = list(set(glob.glob(os.path.join(DATA_DIR, '*.parquet'))))

In [3]:
#filtering function
def createFilter(data,minMu,minEl):
    mu_isolation_cut = ak.sum(abs(data.Muon_pfRelIso04_all) < 0.4, axis=1) == minMu
    muon_pt_cut = ak.sum(data.Muon_pt > 5, axis=1) ==minMu
    muon_eta_cut = ak.sum(abs(data.Muon_eta) < 2.4, axis=1) == minMu
    muon_sip_cut = ak.sum(abs(data.Muon_sip3d) < 4, axis=1) == minMu
    muon_dxy_cut = ak.sum(abs(data.Muon_dxy) < 0.5, axis=1) == minMu
    muon_dz_cut = ak.sum(abs(data.Muon_dz) < 1.0, axis=1) == minMu
    mu_cuts = mu_isolation_cut * muon_pt_cut * muon_eta_cut * muon_sip_cut * muon_dxy_cut * muon_dz_cut

    el_isolation_cut = ak.sum(abs(data.Electron_pfRelIso03_all) < 0.4, axis=1) == minEl
    el_pt_cut = ak.sum(data.Electron_pt > 7, axis=1) ==minEl
    el_eta_cut = ak.sum(abs(data.Electron_eta) < 2.5, axis=1) == minEl
    el_sip_cut = ak.sum(abs(data.Electron_sip3d) < 4, axis=1) == minEl
    el_dxy_cut = ak.sum(abs(data.Electron_dxy) < 0.5, axis=1) == minEl
    el_dz_cut = ak.sum(abs(data.Electron_dz) < 1.0, axis=1) == minEl
    el_cuts = el_isolation_cut * el_pt_cut * el_eta_cut * el_sip_cut * el_dxy_cut * el_dz_cut

    return mu_cuts * el_cuts

In [4]:
variable_dic={}
for inf in PATHS:

    #reading the files
    data=ak.from_parquet(inf)

    # E and mu impact parameters and their significance
    data['Muon_ip3d']=np.sqrt(data.Muon_dxy*data.Muon_dxy + data.Muon_dz*data.Muon_dz)
    data['Muon_sip3d']=data.Muon_ip3d/np.sqrt(data.Muon_dxyErr*data.Muon_dxyErr + data.Muon_dzErr*data.Muon_dzErr)
    data['Electron_ip3d']=np.sqrt(data.Electron_dxy*data.Electron_dxy + data.Electron_dz*data.Electron_dz)
    data['Electron_sip3d']=data.Electron_ip3d/np.sqrt(data.Electron_dxyErr*data.Electron_dxyErr + data.Electron_dzErr*data.Electron_dzErr)

    name=re.search("(\w*).parquet", inf)
    
    namepath1="hzz4mu"
    namepath2="hzz2e2mu"
    namepath3="hzz4e"
    
    #using the filtering function
    fourMu=createFilter(data,4,0)
    twoMuTwoE=createFilter(data,2,2)
    fourEl=createFilter(data,0,4)
    
    variable_dic[namepath1+"."+name.group()]=data[fourMu]#.to_list()
    variable_dic[namepath2+"."+name.group()]=data[twoMuTwoE]#.to_list()
    variable_dic[namepath3+"."+name.group()]=data[fourEl]#.to_list()

    # Let's make arrays for the three final states    
    #hzz4mu=data[fourMu]#.to_list()
    #hzz2e2mu=data[twoMuTwoE]#.to_list()
    #hzz4e=data[fourEl]#.to_list()
    
    
    # Sample sizes:
    print(name.group()+ " Higgs to ZZ to 4 muons sample size: %i" % len(variable_dic[namepath1+"."+name.group()]))
    print(name.group()+ " Higgs to ZZ to 2 electrons and 2 muons sample size: %i" % len(variable_dic[namepath2+"."+name.group()]))
    print(name.group()+ " Higgs to ZZ to 4 electrons sample size: %i\n" % len(variable_dic[namepath3+"."+name.group()]))

Run2012C_DoubleMuParked.parquet Higgs to ZZ to 4 muons sample size: 142
Run2012C_DoubleMuParked.parquet Higgs to ZZ to 2 electrons and 2 muons sample size: 125
Run2012C_DoubleMuParked.parquet Higgs to ZZ to 4 electrons sample size: 0

ZZTo4e.parquet Higgs to ZZ to 4 muons sample size: 0
ZZTo4e.parquet Higgs to ZZ to 2 electrons and 2 muons sample size: 0
ZZTo4e.parquet Higgs to ZZ to 4 electrons sample size: 70068

Run2012C_DoubleElectron.parquet Higgs to ZZ to 4 muons sample size: 0
Run2012C_DoubleElectron.parquet Higgs to ZZ to 2 electrons and 2 muons sample size: 97
Run2012C_DoubleElectron.parquet Higgs to ZZ to 4 electrons sample size: 48

skim.parquet Higgs to ZZ to 4 muons sample size: 0
skim.parquet Higgs to ZZ to 2 electrons and 2 muons sample size: 77306
skim.parquet Higgs to ZZ to 4 electrons sample size: 0

Run2012B_DoubleElectron.parquet Higgs to ZZ to 4 muons sample size: 0
Run2012B_DoubleElectron.parquet Higgs to ZZ to 2 electrons and 2 muons sample size: 57
Run2012B_Doub

In [5]:
data=variable_dic['hzz4mu.SMHiggsToZZTo4L.parquet']
data2=variable_dic['hzz4e.SMHiggsToZZTo4L.parquet']
data3=variable_dic['hzz2e2mu.SMHiggsToZZTo4L.parquet']

In [6]:
data[0]

<Record ... Electron_sip3d: []} type='{"run": int32, "luminosityBlock": uint32, ...'>

### Only Electron interactions

In [7]:
start=dt.datetime.now()
massMuon = []
count = 0
for e in data:
    nMuon = e['nMuon'] # How many compnents in muon vector
    mus = []
    for i in range(nMuon):
        mus.append(vector.obj(pt=e['Muon_pt'][i],eta=e['Muon_eta'][i],
                              phi=e['Muon_phi'][i],mass=e['Muon_mass'][i])) # list of vectors
        
    combos = itertools.combinations(range(nMuon),2) # find combinations like (0, 1), (0, 2),...,(2, 3)
    for j in combos:                                # in total there are 6 possible combinations bc of 4 comp
        m1 = mus[j[0]] # 1st mass of the combination
        m2 = mus[j[1]] # 2nd mass of the combination
        z = m1 + m2              # sum of the muon masses 
        ch1 = e['Muon_charge'][j[0]] # 1st charge of the combination
        ch2 = e['Muon_charge'][j[1]] # 2nd charge of the combination
        if not (ch1 + ch2): # '+/-' and '+/-' not possible, only '+' and '-'
            count = count + 1
            massMuon.append(z.mass) # find leptons with different charge combinations 
                                 # and write into a list named 'massMuon'

stop=dt.datetime.now()
print(count)      # how many components are in massMuon
print(massMuon[0:5]) # print first 5 sums of masses for show
print(stop-start)


34895
[7.725514161258727, 55.01387254537647, 21.10976537457914, 93.66340214757945, 58.027073599155536]
0:00:41.968555


In [8]:
# Function to find element
# closest to given target.
# LINK: https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
# Returns element closest to target in array
def find_nearest(array, target):
    array = np.asarray(array)
    idx = (np.abs(array - target)).argmin()
    return array[idx]

In [9]:
find_nearest(massMuon, 93)

93.00144433793005

### Only Electron Interactions

In [10]:
start=dt.datetime.now()
massElec = []
count = 0
for e in data2:
    nElec = e['nElectron'] # How many compnents in muon vector
    elec = []
    for i in range(nElec):
        elec.append(vector.obj(pt=e['Electron_pt'][i],eta=e['Electron_eta'][i],
                              phi=e['Electron_phi'][i],mass=e['Electron_mass'][i])) # list of vectors
        
    combos = itertools.combinations(range(nElec),2) # find combinations like (0, 1), (0, 2),...,(2, 3)
    for j in combos:                                # in total there are 6 possible combinations bc of 4 comp
        m1 = elec[j[0]] # 1st mass of the combination
        m2 = elec[j[1]] # 2nd mass of the combination
        z = m1 + m2              # sum of the muon masses 
        ch1 = e['Electron_charge'][j[0]] # 1st charge of the combination
        ch2 = e['Electron_charge'][j[1]] # 2nd charge of the combination
        if not (ch1 + ch2): # '+/-' and '+/-' not possible, only '+' and '-'
            count = count + 1
            massElec.append(z.mass) # find leptons with different charge combinations 
                                 # and write into a list named 'massElec'

stop=dt.datetime.now()
print(count)      # how many components are in massElec
print(massElec[0:5]) # print first 5 sums of masses for show
print(stop-start)

22719
[76.75253069603737, 22.80629366154358, 19.941291162775382, 39.070824593451945, 48.75668471208008]
0:00:26.033499


In [11]:
find_nearest(massElec, 93)

93.00005069277508

### Electron and Muon Interactions

In [12]:
start=dt.datetime.now()
massElMu = []
count = 0
for e in data3:
    nElMu = e['nElectron'] + e['nMuon']# How many compnents in muon vector
    elmu = []
    for i in range(nElMu):
        elmu.append(vector.obj( pt   = [y for x in (e['Electron_pt'][i],e['Muon_pt'][i]) for y in x],
                                eta  = [y for x in (e['Electron_eta'][i],e['Muon_eta'][i]) for y in x],
                                phi  = [y for x in (e['Electron_phi'][i],e['Muon_phi'][i]) for y in x],
                                mass = [y for x in (e['Electron_mass'][i],e['Muon_mass'][i]) for y in x]
                                )) # cobining 
        
    combos = itertools.combinations(range(nElMu),2) # find combinations like (0, 1), (0, 2),...,(2, 3)
    for j in combos:                                # in total there are 6 possible combinations bc of 4 comp
        m1 = elmu[j[0]] # 1st mass of the combination
        m2 = elmu[j[1]] # 2nd mass of the combination
        z = m1 + m2              # sum of the muon masses 
        mu_ch1 = e['Muon_charge'][j[0]] # 1st charge of the combination
        mu_ch2 = e['Muon_charge'][j[1]] # 2nd charge of the combination
        if not (ch1 + ch2): # '+/-' and '+/-' not possible, only '+' and '-'
            count = count + 1
            massElMu.append(z.mass) # find leptons with different charge combinations 
                                 # and write into a list named 'massElec'

stop=dt.datetime.now()
print(count)      # how many components are in massElec
print(massElMu[0:5]) # print first 5 sums of masses for show
print(stop-start)

TypeError: 'float' object is not iterable